In [1]:
#get anndata object from util file 
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import squidpy as sq
import numpy as np
import matplotlib.pyplot as plt
import skimage
import anndata as ad
import pandas as pd
import numba
from numba import jit

In [2]:
def return_anndata(path):
    
    img = skimage.io.imread(path)
    ski_img = skimage.measure.regionprops(img)
    df = pd.DataFrame(ski_img)
    var_array = df.head(1).values
    list_var = list(var_array.flatten())
    labels = len(ski_img)
    df = pd.DataFrame(index = list_var, columns = np.arange(1,labels,1))

    
    for ind in df.index:
        for val in df.columns:
            df.loc[ind,val] = getattr(ski_img[val],ind)
            
    for val in df.columns:
        df.loc['bbox',val] = np.asarray(df.loc['bbox',val]).reshape(1,4)
        
        
    for val in df.columns:
        df.loc['centroid',val] = np.asarray(df.loc['centroid',val]).reshape(1,2)
        
        
    for ind in df.index:
        for val in df.columns:
            if type(df.loc[ind,val]) == np.ndarray:
                df.loc[ind,val] = df.loc[ind,val][~np.isnan(df.loc[ind,val])].flatten()
                
                
                
    scalar_list = ['area','area_bbox','area_convex','area_filled','axis_major_length','axis_minor_length',
            'eccentricity', 'equivalent_diameter_area','euler_number','extent','feret_diameter_max',
              'label','orientation','perimeter','perimeter_crofton','solidity'] 
    scalar_df = df.loc[['area','area_bbox','area_convex','area_filled','axis_major_length','axis_minor_length',
            'eccentricity', 'equivalent_diameter_area','euler_number','extent','feret_diameter_max',
              'label','orientation','perimeter','perimeter_crofton','solidity']]
                   
    multidim_list = ['bbox','centroid','centroid_local', 'coords', 'image', 'image_convex', 'image_filled',
                'inertia_tensor', 'inertia_tensor_eigvals', 'moments', 'moments_central' ,'moments_hu',
                'moments_normalized']
    multidim_df = df.loc[['bbox','centroid','centroid_local', 'coords', 'image', 'image_convex', 'image_filled',
                'inertia_tensor', 'inertia_tensor_eigvals', 'moments', 'moments_central' ,'moments_hu',
                'moments_normalized']]
    
    

    mt = multidim_df.transpose()
    
    X = scalar_df.T.values
    adata = ad.AnnData(X)
    adata.var_names = scalar_df.index
    adata.obs_names = scalar_df.columns
    adata.var = pd.DataFrame(index = scalar_list, columns = scalar_list)
    adata.obs = pd.DataFrame(index = scalar_df.columns, columns = scalar_df.columns)
    for col in mt.columns:
        adata.obsm[col] = mt[col].values
        
    i = 0
    for row in multidim_list:
        adata.uns[row] = multidim_df.iloc[[i]].values.flatten()
        i = i+1
        
    return adata
    

In [3]:
#loading dictionary 
import pickle 
with open('mean_intensities.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [4]:
#key convert
index_558 = ['LMNB1_8028', 'LMNB1_8028','LMNB1_8028','LMNB1_8028']
index_610 = ['ELAVL2','IBA1','PVALB', 'PDGFRA']
index_687 = ['GFAP','MAP2','NFH','MBP']

delete_entries = ['one_z_plane_558_2_8028', 'one_z_plane_558_3_8028','one_z_plane_558_4_8028']

del loaded_dict['one_z_plane_558_2_8028']
del loaded_dict['one_z_plane_558_3_8028']
del loaded_dict['one_z_plane_558_4_8028']

loaded_dict['LMNB1'] = loaded_dict.pop('one_z_plane_558_1_8028')
loaded_dict['ELAVL2'] = loaded_dict.pop('one_z_plane_610_1_8028')
loaded_dict['IBA1'] = loaded_dict.pop('one_z_plane_610_2_8028')
loaded_dict['PVALB'] = loaded_dict.pop('one_z_plane_610_3_8028')
loaded_dict['PDGFRA'] = loaded_dict.pop('one_z_plane_610_4_8028')
loaded_dict['GFAP'] = loaded_dict.pop('one_z_plane_687_1_8028')
loaded_dict['MAP2'] = loaded_dict.pop('one_z_plane_687_2_8028')
loaded_dict['NFH'] = loaded_dict.pop('one_z_plane_687_3_8028')
loaded_dict['MBP'] = loaded_dict.pop('one_z_plane_687_4_8028')

In [5]:
def del_nan(arr):
    del_list = []
    mask = np.isnan(arr)
    for m,v in zip(mask, range(len(mask))):
        if m == True:
            del_list.append(v)
        
    arr = np.delete(arr,del_list)
    return(arr)
    
    

In [6]:
for key in loaded_dict.keys():
    val = loaded_dict[key]
    val = del_nan(val)
    #val = np.roll(val,-1)
    loaded_dict.update({key:val[1:]})
        

In [7]:
ad_object   = return_anndata('/gpfs/commons/groups/nygcfaculty/PySeq/20210428_mouse_genotype_2/segmented_sections/m387ntga2_labels.tiff')

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:875: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [1, 2, 3, 4, 5]

    Inferred to be: integer

  names = self._prep_dim_index(names, "obs")
/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:801: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [1, 2, 3, 4, 5]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [8]:
mat = ad_object.X

In [9]:
for key in loaded_dict.keys():
    mat = np.append(mat, np.asarray([loaded_dict[key]]).transpose(), axis = 1)
    

In [24]:
#remake new anndata object
adt = ad.AnnData(mat)

In [25]:
lis = ad_object.var_names
var = list(lis)
post_var = list(loaded_dict.keys())
var.extend(post_var)

In [26]:
adt.var_names = var
adt.obs_names = np.arange(1,np.shape(mat)[0] + 1,1)
#adt.obsm = ad_object.obsm
adt.uns = ad_object.uns

In [27]:
for j in adt.uns.keys():
    #adt.obsm[j] = np.array2string(adt.obsm[j])
    adt.uns[j] = np.array2string(adt.uns[j])
    

In [28]:
np.shape(ad_object.X)

(11169, 16)

In [29]:
print(adt.obs.columns)

Index([], dtype='object')


In [30]:
adt.obs.columns = adt.obs.columns.astype(str)
adt.var.columns = adt.var.columns.astype(str)
adt.write('/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m387ntga2_labels_anndata_sample.h5ad')

In [20]:
adt.uns['bbox'] = np.array2string(adt.uns['bbox'])

AttributeError: 'str' object has no attribute 'size'

In [130]:
add = ad.read_h5ad('/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m387ntga2_labels_anndata_sample.h5ad')

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [133]:
adt.write_zarr('/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m387ntga2_labels_anndata_sample.zarr')

In [19]:
######### ROUGH WORK #####
for j in adt.uns.keys():
    #adt.obsm[j] = np.array2string(adt.obsm[j])
    adt.uns[j] = adt.uns[j].tobytes()
    

AttributeError: 'bytes' object has no attribute 'tobytes'

In [4]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

dask==2022.5.2
matplotlib==3.5.2
numpy==1.21.5
pandas==1.4.3
squidpy==1.2.2
numba==0.55.1
anndata==0.7.8
